In [1]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import PreTrainedTokenizer


c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

dataset = load_dataset('csv', data_files='content_gsheet_info.csv', split='train')


Generating train split: 7968 examples [00:00, 39830.45 examples/s]


In [3]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [4]:
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer
from vertexai.preview.language_models import TextEmbeddingModel

CORE_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(CORE_MODEL_NAME)
#tokenized_col_A = tokenizer(col_A_txt, padding=True, truncation=True, return_tensors='pt')
#tokenized_col_B = tokenizer(col_B_txt, padding=True, truncation=True, return_tensors='pt')

# Initialize the model
model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")

OSError: textembedding-gecko-multilingual is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
def preprocess_function(examples):
    course_name_description = examples['course_name_description']
    principal_ability_description = examples['principal_ability_description']
    second_ability_description = examples['second_ability_description']

    # Concatenate the columns
    input_text = course_name_description + " " + principal_ability_description + " " + second_ability_description

    # Tokenize the input text
    inputs = tokenizer(input_text, padding='max_length', truncation=True, max_length=512)

    return inputs

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
def custom_loss(model, inputs, labels):
    outputs, embeddings = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Compute the loss for the course_name_description and principal_ability_description embeddings
    loss_fct = torch.nn.MSELoss()
    loss_course_name_description = loss_fct(embeddings[:, 0], embeddings[:, 1].detach())

    # Compute the loss for the second_ability_description and principal_ability_description embeddings
    loss_second_ability_description = loss_fct(embeddings[:, 2], embeddings[:, 1].detach())

    # Combine the losses
    loss = loss_course_name_description + loss_second_ability_description

    return loss

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
#from vertexai.preview.language_models import TextEmbeddingModel
#model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")

In [ ]:
from transformers import TextEmbeddingModel, TextEmbeddingModelTokenizer

model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")
tokenizer = TextEmbeddingModelTokenizer.from_pretrained('textembedding-gecko-multilingual')

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='path/to/output/directory',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir='path/to/logging/directory',
    logging_steps=1000,
    save_steps=1000,
    evaluation_strategy='epoch',
    compute_metrics=lambda pred: {'accuracy': accuracy_score(pred.label_ids, pred.predictions.argmax(-1))},
    label_smoothing_factor=0.1,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    fp16=True,
    fp16_opt_level="O1",
    gradient_accumulation_steps=1,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_first_step=True,
    dataloader_num_workers=2,
    gradient_checkpointing=True,
    gradient_checkpointing_steps=1000,
    label_names=['label'],
    eval_accumulation_steps=10,
    push_to_hub=False,
    report_to='none',
    load_job_checkpoint=False,
    metric_for_checkpointing='accuracy',
    greater_is_better=True,
    save_strategy='steps',
    save_total_limit=2,
    save_steps=1000,
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    lr_warmup_steps=1000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('path/to/fine-tuned/model')